In [7]:
import os
current_dir = os.getcwd()
folder_path = current_dir + "\\20_newsgroups"
newsgroups = os.listdir(folder_path)

In [8]:
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [9]:
import re
vocab = {}
for groups in newsgroups:
    group_path = folder_path + "\\" + groups
    articles = os.listdir(group_path)
    for article in articles:
        article_path = group_path + "\\" + article
        with open(article_path) as file:
            content = file.read()
            clean_content = re.sub(r'[^A-Za-z\s]', ' ', content)
            words = clean_content.split()
            for i in words:
                i = i.lower()
                if i in stop_words or len(i) <= 2:
                    continue
                else:
                    if i in vocab:
                        vocab[i]+=1
                    else:
                        vocab[i]=1

In [10]:
sorted_vocab = dict(sorted(vocab.items(), key=lambda item: item[1], reverse=True))

In [11]:
features = {}
k = 0
for i in sorted_vocab:
    features[i] = k
    k+=1
    if k == 1000:
        break

In [12]:
X = []
Y = []
for groups in newsgroups:
    group_path = folder_path + "\\" + groups
    articles = os.listdir(group_path)
    for article in articles:
        article_path = group_path + "\\" + article
        d = np.zeros(1000)
        with open(article_path) as file:
            content = file.read()
            clean_content = re.sub(r'[^A-Za-z\s]', ' ', content)
            words = clean_content.split()
            for i in words:
                if i in features:
                    d[features[i]]+=1
        X.append(d)
        Y.append(groups)

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB 
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [20]:
print(model.score(X_test,y_test), model.score(X_train,y_train))

0.8511666666666666 0.8797599485604058


In [50]:
def fit(x_train, y_train):
    result = {}
    class_values = set(y_train)
    for current_class in class_values:
        result[current_class] = {}
        result['total_data'] = len(y_train)
        current_class_rows = y_train == current_class
        x_train_current = x_train[current_class_rows]
        y_train_current = y_train[current_class_rows]
        num_features = x_train.shape[1]
        result[current_class]['total_count'] = len(x_train_current)
        for i in range(1, num_features+1):
            result[current_class][i] = {}
            all_possible_values = set(x_train_current[:,i-1])
            for feature in all_possible_values:
                result[current_class][i][feature] = (x_train_current[:,i-1] == feature).sum()
    return result

In [61]:
def probability(dictionary, x, current_class):
    output = np.log(dictionary[current_class]['total_count']) - np.log(dictionary['total_data'])
    num_features = len(dictionary[current_class].keys()) - 1
    for i in range(1, num_features+1):
        X = x[i-1]
        count_current_class_with_value_X = dictionary[current_class][i].get(X,0)
        count_current_class = dictionary[current_class]['total_count'] + len(dictionary[current_class][i].keys())
        prob = np.log(count_current_class_with_value_X + 1) - np.log(count_current_class) 
        output = output + prob
    return output

In [66]:
def predictSinglePoint(dictionary, x):
    classes = dictionary.keys()
    best_p = -1000
    best_class = -1
    first_run = True
    for current_class in classes:
        if(current_class == 'total_data'):
            continue
        p_current_class = probability(dictionary, x, current_class)
        if (first_run or p_current_class > best_p):
            best_p = p_current_class
            best_class = current_class
        first_run = False   
    return best_class

In [53]:
def predict(dictionary, x_test):
    y_pred = []
    for x in x_test:
        x_class = predictSinglePoint(dictionary, x)
        y_pred.append(x_class)
        
    return y_pred

In [54]:
def score(y_pred, y_test):
    return np.mean(y_pred == y_test)

In [58]:
X = np.array(X)
Y = np.array(Y)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.05, random_state=42)

In [59]:
dict_fit = fit(X_train, y_train)

In [67]:
y_pred = predict(dict_fit, X_test)

In [68]:
print(score(y_pred,y_test))

0.93
